# Debugging in DSC

## Module executable and generated script

As seen from previous tutorials, the [module executables in DSC](../tutorials/dsc_result.html) are in fact incomplete due to lack of input data and parameter specifications to run. Under the hood, DSC generates R, Python or Shell scripts based on module executable scripts, adding to it specification of module parameters and pipeline variables, then executes the complete script. This generated script is subject to two types of errors:

1. Inconsistency between DSC interface and module executables
2. Error from module executables as provided
 
In the presence of errors DSC will fail. However the automatically generated script will be saved to disk and will be pointed to you as part of the error prompt on the command console so that you can reproduce and fix the bugs. 

Here we look at [this toy example](https://github.com/stephenslab/dsc/tree/master/vignettes/one_sample_location_error). We undermine the setup to demonstrate the 2 types of errors we have discussed.

## Buggy DSC interface

To demonstrate inconsistency between interface and module we change the first block of DSC script from:

```
normal: R(x <- rnorm(n,mean = mu,sd = 1))
  mu: 0
  n: 100
  $data: x
  $true_mean: mu
```

to:

```
normal: R(x <- rnorm(n,mean = mu,sd = 1))
  nu: 0
  n: 100
  $data: x
  $true_mean: mu
```

creating a parameter `nu`, a typo of `mu`, that does not exist in `normal` module `rt.R`. Consequently the required `mu` variable for these routines will not be provided by DSC interface. We save this configuration file as `first_error_v1.dsc` and run it:

In [3]:
dsc first_error_v1.dsc -o dsc_result

INFO: Checking R library dscrutils@stephenslab/dsc/dscrutils ...
INFO: DSC script exported to dsc_result.html
INFO: Constructing DSC from first_error_v1.dsc ...
INFO: Building execution graph & running DSC ...
DSC:  53%|████████████████████▊                  | 8/15 [00:03<00:02,  2.54it/s]
ERROR: [566b788d-73de-4e34-85ed-b480227a3e5d]: [normal_0]: 
---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
script_-3186687848203597564 in <module>
      
      
----> """, workdir = './', stderr = f"{_output:n}.stderr", stdout = f"{_output:n}.stdout")

RuntimeError: Failed to execute commmand "Rscript /tmp/tmp9arx3lf5.R" (ret=1, workdir=/home/gaow/Documents/TempDir/dsc/vignettes/one_sample_location, script now in /home/gaow/Documents/TempDir/dsc/vignettes/one_sample_location/.sos/normal_0_0_a467d5c4.R, err=dsc_result/normal/normal_1.stderr)
[DSC]: 7 pending steps: DSC_0, a_abs_err, b_sq_err, c_abs

: 1

The error message is a bit wordy but the part most relevant to debugging are highlighted in green color on your terminal. In this case multiple generated scripts have failed, but the cause is likely the same. You can try to resolve the problem starting from the last error message and see if the fix will help with other errors.

You can also see that a file called `dsc_result.scripts.html` is generated. It bundles generated scripts from all module instances involved in the benchmark up to the point the errors occur.

### Debug with the problematic script
As prompted in the error message, let's see what's inside `dsc_result/normal/normal_1.stderr`:

In [4]:
cat dsc_result/normal/normal_1.stderr

Error in rnorm(n, mean = mu, sd = 1) : object 'mu' not found
Execution halted


Okey so what's going on? Let's take a look at this script:

In [6]:
cat .sos/normal_0_0_a467d5c4.R

## r script UUID: 89241936

dscrutils:::empty_text(c("dsc_result/normal/normal_1.stdout", "dsc_result/normal/normal_1.stderr"))
DSC_REPLICATE <- 1
n <- 100
nu <- 0
TIC_C1997B87 <- proc.time()
set.seed(DSC_REPLICATE + 89241936)

## BEGIN DSC CORE
x <- rnorm(n,mean = mu,sd = 1)
## END DSC CORE

saveRDS(list(data=x, true_mean=mu, DSC_DEBUG=list(time=proc.time() - TIC_C1997B87, script=dscrutils:::load_script(), replicate=DSC_REPLICATE, session=toString(sessionInfo()))), 'dsc_result/normal/normal_1.rds')
dscrutils:::rm_if_empty(c("dsc_result/normal/normal_1.stdout", "dsc_result/normal/normal_1.stderr"))




It seems like the line `x <- rnorm(n,mean = mu,sd = 1)
` looks for variable `mu`, which was not previously defined in the script. We know from the [DSC script](https://github.com/stephenslab/dsc/blob/master/vignettes/one_sample_location_error/first_error_v1.dsc) that we had set `nu` not `mu`. So obviously there is a glitch on the interface which can be fixed by changing `nu` to `mu` in the DSC script.

In practice when you run into problems that you cannot immediately tell what is going on from `err` file or script file, you should load the problematic script `.sos/normal_0_0_a467d5c4.R` to some interactive session, try to debug and fix the script from there (do not worry about changing this script), test until it works. Then you should have learned where the problem is, and you can apply what you've learned to patching DSC script or the original module code. *Fixing these generated scripts is not enough as they will be overwritten next time DSC runs*.

## Buggy module script

Now let's undermine the module code instead. We change from:

```
mean: R(y <- mean(x))
```

to 

```
mean: R(y <- "meow")
```

As you can probably tell, this will not result an error for this module, but may impact downstream ones. So let's find it out!

In [13]:
dsc first_error_v2.dsc -o dsc_result

INFO: DSC script exported to dsc_result.html
INFO: Constructing DSC from first_error_v2.dsc ...
INFO: Building execution graph & running DSC ...
DSC:  93%|███████████████████████████████████▍  | 14/15 [00:03<00:00,  4.19it/s]
ERROR: [317e740e-7dea-478f-ae3d-cfec17674a99]: [abs_err_0]: 
---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
script_-6533540462407766879 in <module>
      
      
----> """, workdir = './', stderr = f"{_output:n}.stderr", stdout = f"{_output:n}.stdout")

RuntimeError: Failed to execute commmand "Rscript /tmp/tmpf1phv0qi.R" (ret=1, workdir=/home/gaow/Documents/TempDir/dsc/vignettes/one_sample_location, script now in /home/gaow/Documents/TempDir/dsc/vignettes/one_sample_location/.sos/abs_err_0_0_909abdf6.R, err=dsc_result/abs_err/normal_1_mean_1_abs_err_1.stderr)
[ffc0b82c-f9da-42bf-ac5b-99141a1bcaa2]: [sq_err_0]: 
-------------------------------------------------

: 1

So there are multiple module failures. In case like this we should check and fix the first one first. Same as before, we check:

In [14]:
cat dsc_result/abs_err/normal_1_mean_1_abs_err_1.stderr

Error in x - y : non-numeric argument to binary operator
Execution halted


It is not immediately clear what is going on, so we examine the code `.sos/abs_err_0_0_909abdf6.R` by adding a couple of `print` commands before the line in question:

In [18]:
## r script UUID: 90107557
dscrutils:::empty_text(c("dsc_result/abs_err/normal_1_mean_1_abs_err_1.stdout", "dsc_result/abs_err/normal_1_mean_1_abs_err_1.stderr"))
DSC_A0BFA20F <- list()
input.files <- c('dsc_result/normal/normal_1.rds','dsc_result/mean/normal_1_mean_1.rds')
for (i in 1:length(input.files)) DSC_A0BFA20F <- dscrutils:::merge_lists(DSC_A0BFA20F, readRDS(input.files[i]))
DSC_REPLICATE <- DSC_A0BFA20F$DSC_DEBUG$replicate
x <- DSC_A0BFA20F$est_mean
y <- DSC_A0BFA20F$true_mean
TIC_A0BFA20F <- proc.time()
set.seed(DSC_REPLICATE + 90107557)

## BEGIN DSC CORE

print(x)
print(y)
e <- abs(x - y)
## END DSC CORE

saveRDS(list(error=e, DSC_DEBUG=list(time=proc.time() - TIC_A0BFA20F, script=dscrutils:::load_script(), replicate=DSC_REPLICATE, session=toString(sessionInfo()))), 'dsc_result/abs_err/normal_1_mean_1_abs_err_1.rds')
dscrutils:::rm_if_empty(c("dsc_result/abs_err/normal_1_mean_1_abs_err_1.stdout", "dsc_result/abs_err/normal_1_mean_1_abs_err_1.stderr"))

[1] "meow"
[1] 0


ERROR: Error in x - y: non-numeric argument to binary operator


It looks like `x`, instead of being a number, is a string `meow`. Therefore the line `e <- abs(x - y)` is doomed to fail. We know from the DSC script that `x` comes from an upstream module group `analyze`, that somehow returned `meow` as output. We can also see from this script that the variable comes from one of `'dsc_result/normal/normal_1.rds','dsc_result/mean/normal_1_mean_1.rds'`. From our knowledge of the DSC script we know it should come from `'dsc_result/mean/normal_1_mean_1.rds'` the `mean` module. Therefore there is nothing we can do in the current script to fix the problem.

### Debugging with upstream scripts

Now we want to find out what has produced the problematic `x` variable. That is, we want to find the module instance that generated `'dsc_result/mean/normal_1_mean_1.rds'` from which `x` is loaded. However the problematic script above did not tell us where this data comes from. This leads us to examine other scripts in the benchmark that leads to the error, in [`dsc_result.scripts.html`](../tutorials/dsc_result.scripts_e2.html).

You can open up [this file](../tutorials/dsc_result.scripts_e2.html) in your web browser, and *search* (`ctrl-F`) for the phrase `dsc_result/mean/normal_1_mean_1.rds`. This leads you to the following: 

![debug-1](../img/debug-tips-1.png)

Unsurprisingly, running these code by itself produces no error message -- this is expected because otherwise DSC would have failed at this stage. However we know from previous investigation that the output does not look right. For this example we can spot instantly that the `meow` string somehow &#128049; replaced the code for location parameter estimation; and we can fix it easily. In practice, one may need to load both this code chunk and the previous `.sos/abs_err_0_0_909abdf6.R` to an interactive session, fix it until `.sos/abs_err_0_0_909abdf6.R` works, and apply the knowledge learned to the original module line `mean: R(y <- "meow")` in the DSC script.

## Debug with `--target` and `--truncate`
It is highly recommanded that these options be used to check the correctness of DSC benchmark as you develop them. Do not run the entire benchmark unless you have checked it bit by bit. `--truncate` will only run one instance from a module, thus enables running a fraction of DSC benchmark relatively quickly to ensure everthing is correct. Additionally it is suggested that a benchmark be executed module by module using `--target`: the next module or module ensemble is only attedmpted when there is no obvious problem with the previous module. For example for a DSC benchmark:

```
DSC:
  define: 
      simulate: normal, t
      estimate: median, mean
  run: simulate * estimate * mse
```
When the benchmark is tested for the first time, one should at least run the following sequentially:

```
dcs file.dsc --target normal --truncate
dcs file.dsc --target t --truncate
dcs file.dsc --target "normal * mean" --truncate
dcs file.dsc --target "normal * median" --truncate
dcs file.dsc --target "normal * median * mse" --truncate
```

This will ensure all module are tested for at least one instance. Then you can run the benchmark knowing that the chances of errors has already been reduced.

```
dcs file.dsc 
```

## More output info
The run-time information are kept in greater when running DSC with verbosity level greater than `2`. Though typically unnecessary, if you still cannot fix issues at this point it may help re-running benchmark with `-v 4` and look into the screen output (or, redirect them to a file) for more run-time information to help performing diagnostics.